In [105]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import re
import json
import ast
import string

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', 0)


In [42]:
df_test = pd.read_csv("/data3/abraham/asr_benchmarking/data/masakhaner_intron_prod_test_quality.csv")
df_test

,date,audio_id,audio_path,audio_duration,sentence_id,source,doc_id,seq_id,text,prediction,...,Music,Good Quality,has_entity,PER,LOC,ORG,DATE,entities,entities_group,entities_others
0,2024-02-28 00:12:58.764668,513ef5d4-a823-4113-b757-4ed91d5d5e17,/data4/data/prod2/blob_52537.wav,1.0,7e3ff9d7-79fe-4eba-9c0b-d5c10074a2f5,Production-Test-Set,1107,1107,"Hello, my name is Tolu. I'm an orthodontist in a tertiary hospital in Nigeria. I deal in correcting malocclusions in children and adults and giving them aesthetic smiles. It's been quite a rewarding experience. seeing children leaving my clinic with smiles on their faces and seeing their parents beaming, knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards. It's a fun job and I usually am satisfied at the end of the day. Also one of the most highly rewarding specialties in dentistry. But in Nigeria and other parts of the world, I would like you to come visit us one of these days. Thank you for listening.",NaN,...,NaN,NaN,1,['Tolu'],"['Nigeria', 'Nigeria']",NaN,NaN,"[{""entity"": ""B-PER"", ""score"": 0.9999862909317017, ""index"": 6, ""word"": ""\u2581To"", ""start"": 18, ""end"": 20}, {""entity"": ""I-PER"", ""score"": 0.9974403381347656, ""index"": 7, ""word"": ""lu"", ""start"": 20, ""end"": 22}, {""entity"": ""B-LOC"", ""score"": 0.9999977350234985, ""index"": 23, ""word"": ""\u2581Nigeria"", ""start"": 70, ""end"": 77}, {""entity"": ""B-LOC"", ""score"": 0.9999985694885254, ""index"": 141, ""word"": ""\u2581Nigeria"", ""start"": 589, ""end"": 596}]","[{""entity_group"": ""PER"", ""score"": 0.9987133145332336, ""word"": ""Tolu"", ""start"": 18, ""end"": 22}, {""entity_group"": ""LOC"", ""score"": 0.9999977350234985, ""word"": ""Nigeria"", ""start"": 70, ""end"": 77}, {""entity_group"": ""LOC"", ""score"": 0.9999985694885254, ""word"": ""Nigeria"", ""start"": 589, ""end"": 596}]",NaN
1,2024-02-28 00:12:58.761194,53f8b873-233f-455d-96e5-bbf5a41205ba,/data4/data/prod2/blob_58081.wav,1.0,056d8c3e-06a6-46da-a0fb-f9d4cad5dd65,Production-Test-Set,1106,1106,Add comma.,NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-02-28 00:12:58.756045,32896923-f849-4aa3-9eda-dccc7284a309,/data4/data/prod2/blob_312.wav,1.0,05f2bfd7-822d-4658-ae31-eaa078438fb5,Production-Test-Set,1105,1105,"Hi, what are you doing today?",NaN,...,NaN,NaN,1,NaN,NaN,NaN,['today'],"[{""entity"": ""B-DATE"", ""score"": 0.9999139308929443, ""index"": 7, ""word"": ""\u2581today"", ""start"": 23, ""end"": 28}]","[{""entity_group"": ""DATE"", ""score"": 0.9999139308929443, ""word"": ""today"", ""start"": 23, ""end"": 28}]",NaN
3,2024-02-28 00:12:58.750599,1e5a3d4f-b2a3-4596-8dff-e266a45c11bb,/data4/data/prod2/blob_68923.wav,1.0,af12a172-5958-4a13-9842-e75d8761a398,Production-Test-Set,1104,1104,"Hypertension generally can cause a lot of diseases. The person could be prone to stroke, the person could also be prone to other diseases like diabetes, diabetes mellitus, precisely type 1, type 2. There could be some neurological disorders, it could be some nephrological disorders, and most times patients that have the heart diseases they most likely come down stroke, meaning they'll be bedridden for a while and have sores.",NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-02-28 00:12:58.745783,58a1c6e2-93e8-41df-b217-cbe19df77340,/data4/data/prod2/blob_45300.wav,1.0,89af42a8-c52e-45ad-9539-366fc85398f8,Production-Test-Set,1103,1103,"most times, when people do general examination you hear them say something like afebrile, not pale, aneteric, acyanosed, but this doesn't affect everybody. Sometimes, some people have general examination to be normal but then again they are having ill state of mind.",NaN,...,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,2024-02-28 00:12:53.231912,834008ec-e9f8-4e9d-95c3-4201ff441af5,/data4/data/prod2/9322150e-2244-40af-bb14-dc

In [107]:
import re
import jiwer
from whisper.normalizers import EnglishTextNormalizer


clinical = ["General-Clinical", "Clinical-Surgery", "Talk-Very-Fast-Clinical", 
            "Pre-Clinical", "Clinical-Medicine", "Pre-Clinical-INT"]
general = ["40yrs-old-and-above", "Talk-Very-Fast-Anyone", 'Transcribe-Inference',
           "Naija-News-Non-Clinical", "News-Anyone-INT", 'Transcribe-Conversation']
legal = ['Legally-Speaking', 'Transcribe-NASS', 'Transcribe-Kenya', 
         'Transcribe-South-Africa', 'Transcribe-Ghana']

inaudible_tags = ['[music] [inaudible]', '(inaudible) ', '[inaudible)', '(inaudible]',
                  '[Inaudible].', '[music]','[INAUDIBLE]',' [Inaudible]', '(Inaudible).',
                  '[Inaudible] ', '[silence]','[Silence]', '[inaudible] ', 'in aduible',
                  '(inaudible)','(Inaudible)','[Inaudible]', 'Inaudible','[inaudible]',
                  '[inaudable]','[Inaudible]','Inaudable ','Blank ', 'inaudible', 'Inaudible ', 
                  '(audio is empty)', 'noise', '(noise)', '[noise]', 'Blank'
                 ]
inaudible_tags_regex = [x.replace('[', '\[').replace(']', '\]').replace('(', '\(').replace(')', '\)') for x in inaudible_tags]
inaudible_tags_joined = "|".join(inaudible_tags_regex)
rx = re.compile(inaudible_tags_joined, re.I)
translator = str.maketrans('', '', string.punctuation)

def clean_filler_words(text):
    text = text.replace("inaudible. ", "").replace("inaudible", "")\
        .replace(" ehm, ", " ").replace(" uh, "," ").replace(" er, "," ").replace("...", " ")
    return text

In [108]:
def clean_text(text):
    """
    post processing to normalized reference and predicted transcripts
    :param text: str
    :return: str
    """
    if type(text) != str:
        print(text)
        return " "

    # remove multiple spaces
    text = clean_filler_words(text)
    text = re.sub(r"\s\s+", " ", text)
    # strip trailing spaces
    text = text.strip()
    text = text.replace('>', '')
    text = text.replace('\t', ' ')
    text = text.replace('\n', '')
    text = text.lower()
    text = text.replace(" comma,", " ") \
        .replace(" koma,", " ") \
        .replace(" coma,", "") \
        .replace(" comma", " ") \
        .replace(" full stop.", " ") \
        .replace(" full stop", " ") \
        .replace(",.", " ") \
        .replace(",,", " ") \
        .replace("?", " ") \
        .replace(",", " ") \
        .replace(".", " ") \
        .replace("  ", " ") \
        .strip()
    text = " ".join(text.split())
    text = text.translate(translator)
    text = re.sub(r"[^a-zA-Z0-9\s\.\,\-\?\:\'\/\(\)\[\]\+\%]", '', text)
    return text

In [109]:
df_test["transcript_original"] = df_test["text"]
df_test["transcript"] = df_test.text.apply(lambda x: clean_text(x) if type(x) == str else "")


In [110]:
# find character that can be used to join the texts
candidate_char = ["_", " ", "-", ";"]
all_chars_in_transcript = set("".join(list(df_test["transcript"].values)))

JOIN_CHAR = None

for char in candidate_char:
    if char in all_chars_in_transcript:
        continue
    else:
        JOIN_CHAR = char
        break

if JOIN_CHAR is None:
    JOIN_CHAR = ""
    
print(JOIN_CHAR)

_


In [111]:
df_test = df_test[df_test['has_entity']==1].copy()
df_test

,date,audio_id,audio_path,audio_duration,sentence_id,source,doc_id,seq_id,text,prediction,...,PER,LOC,ORG,DATE,entities,entities_group,entities_others,transcript_original,transcript,cat_entities
0,2024-02-28 00:12:58.764668,513ef5d4-a823-4113-b757-4ed91d5d5e17,/data4/data/prod2/blob_52537.wav,1.0,7e3ff9d7-79fe-4eba-9c0b-d5c10074a2f5,Production-Test-Set,1107,1107,"Hello, my name is Tolu. I'm an orthodontist in a tertiary hospital in Nigeria. I deal in correcting malocclusions in children and adults and giving them aesthetic smiles. It's been quite a rewarding experience. seeing children leaving my clinic with smiles on their faces and seeing their parents beaming, knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards. It's a fun job and I usually am satisfied at the end of the day. Also one of the most highly rewarding specialties in dentistry. But in Nigeria and other parts of the world, I would like you to come visit us one of these days. Thank you for listening.",NaN,...,['Tolu'],"['Nigeria', 'Nigeria']",NaN,NaN,"[{""entity"": ""B-PER"", ""score"": 0.9999862909317017, ""index"": 6, ""word"": ""\u2581To"", ""start"": 18, ""end"": 20}, {""entity"": ""I-PER"", ""score"": 0.9974403381347656, ""index"": 7, ""word"": ""lu"", ""start"": 20, ""end"": 22}, {""entity"": ""B-LOC"", ""score"": 0.9999977350234985, ""index"": 23, ""word"": ""\u2581Nigeria"", ""start"": 70, ""end"": 77}, {""entity"": ""B-LOC"", ""score"": 0.9999985694885254, ""index"": 141, ""word"": ""\u2581Nigeria"", ""start"": 589, ""end"": 596}]","[{""entity_group"": ""PER"", ""score"": 0.9987133145332336, ""word"": ""Tolu"", ""start"": 18, ""end"": 22}, {""entity_group"": ""LOC"", ""score"": 0.9999977350234985, ""word"": ""Nigeria"", ""start"": 70, ""end"": 77}, {""entity_group"": ""LOC"", ""score"": 0.9999985694885254, ""word"": ""Nigeria"", ""start"": 589, ""end"": 596}]",NaN,"Hello, my name is Tolu. I'm an orthodontist in a tertiary hospital in Nigeria. I deal in correcting malocclusions in children and adults and giving them aesthetic smiles. It's been quite a rewarding experience. seeing children leaving my clinic with smiles on their faces and seeing their parents beaming, knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards. It's a fun job and I usually am satisfied at the end of the day. Also one of the most highly rewarding specialties in dentistry. But in Nigeria and other parts of the world, I would like you to come visit us one of these days. Thank you for listening.",hello my name is tolu im an orthodontist in a tertiary hospital in nigeria i deal in correcting malocclusions in children and adults and giving them aesthetic smiles its been quite a rewarding experience seeing children leaving my clinic with smiles on their faces and seeing their parents beaming knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards its a fun job and i usually am satisfied at the end of the day also one of the most highly rewarding specialties in dentistry but in nigeria and other parts of the world i would like you to come visit us one of these days thank you for listening,tolu_nigeria_nigeria
2,2024-02-28 00:12:58.756045,32896923-f849-4aa3-9eda-dccc7284a309,/data4/data/prod2/blob_312.wav,1.0,05f2bfd7-822d-4658-ae31-eaa078438fb5,Production-Test-Set,1105,1105,"Hi, what are you doing today?",NaN,...,NaN,NaN,NaN,['today'],"[{""entity"": ""B-DATE"", ""score"": 0.9999139308929443, ""index"": 7, ""word"": ""\u2581today"", ""start"": 23, ""end"": 28}]","[{""entity_group"": ""DATE"", ""score"": 0.9999139308929443, ""word"": ""today"", ""start"": 23, ""end"": 28}]",NaN,"Hi, what are you doing today?",hi what are you doing today,today
8,2024-02-28 00:12:58.721021,31465167-a293-4e13-ab6f-d566ab427c1c,/data4/data/prod2/b

In [113]:
def concat_all_entities(row):
    transcript = row['transcript'].lower()
    
    texts = []
    for cat in ['PER', 'LOC', 'ORG', 'DATE']:
        entities = row[cat]
        if type(entities) != float:
            entities = ast.literal_eval(entities)
            texts.extend(entities)
                    
    return f"{JOIN_CHAR}".join(texts).lower()

# These two functions could be merged. 
# They were separated for ease of readability and debugging purposes
def concat_entities(row, category):
    el = row["medical_entities"]
    entities = el["Entities"]
    texts = []
    for entity in entities:
        text = entity["Text"].lower().rstrip("")
        score = entity["Score"]
        entity_category = entity["Category"]

        if category == entity_category and category in VALID_CATEGORIES and score >= SCORE_THRESHOLD:
            if len(text) > 1:
                text= clean_text(text)
                # apply entities corrected in previous step
#                 if text == "paino":
#                     text = text.replace('paino', 'pain')
                
                texts.append(text)

    return f"{JOIN_CHAR}".join(texts)


In [114]:
df_test["cat_entities"] = df_test.apply(lambda x: concat_all_entities(x), axis=1)

In [115]:
df_test["cat_entities"]

0      tolu_nigeria_nigeria     
2      today                    
8      charles_charles_two weeks
9      today                    
11     intron_morning           
            ...                 
956    80 year old              
957    8-year-old               
958    80 year old              
960    80 year old              
961    80 year old              
Name: cat_entities, Length: 533, dtype: object

In [116]:
df_test

,date,audio_id,audio_path,audio_duration,sentence_id,source,doc_id,seq_id,text,prediction,...,PER,LOC,ORG,DATE,entities,entities_group,entities_others,transcript_original,transcript,cat_entities
0,2024-02-28 00:12:58.764668,513ef5d4-a823-4113-b757-4ed91d5d5e17,/data4/data/prod2/blob_52537.wav,1.0,7e3ff9d7-79fe-4eba-9c0b-d5c10074a2f5,Production-Test-Set,1107,1107,"Hello, my name is Tolu. I'm an orthodontist in a tertiary hospital in Nigeria. I deal in correcting malocclusions in children and adults and giving them aesthetic smiles. It's been quite a rewarding experience. seeing children leaving my clinic with smiles on their faces and seeing their parents beaming, knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards. It's a fun job and I usually am satisfied at the end of the day. Also one of the most highly rewarding specialties in dentistry. But in Nigeria and other parts of the world, I would like you to come visit us one of these days. Thank you for listening.",NaN,...,['Tolu'],"['Nigeria', 'Nigeria']",NaN,NaN,"[{""entity"": ""B-PER"", ""score"": 0.9999862909317017, ""index"": 6, ""word"": ""\u2581To"", ""start"": 18, ""end"": 20}, {""entity"": ""I-PER"", ""score"": 0.9974403381347656, ""index"": 7, ""word"": ""lu"", ""start"": 20, ""end"": 22}, {""entity"": ""B-LOC"", ""score"": 0.9999977350234985, ""index"": 23, ""word"": ""\u2581Nigeria"", ""start"": 70, ""end"": 77}, {""entity"": ""B-LOC"", ""score"": 0.9999985694885254, ""index"": 141, ""word"": ""\u2581Nigeria"", ""start"": 589, ""end"": 596}]","[{""entity_group"": ""PER"", ""score"": 0.9987133145332336, ""word"": ""Tolu"", ""start"": 18, ""end"": 22}, {""entity_group"": ""LOC"", ""score"": 0.9999977350234985, ""word"": ""Nigeria"", ""start"": 70, ""end"": 77}, {""entity_group"": ""LOC"", ""score"": 0.9999985694885254, ""word"": ""Nigeria"", ""start"": 589, ""end"": 596}]",NaN,"Hello, my name is Tolu. I'm an orthodontist in a tertiary hospital in Nigeria. I deal in correcting malocclusions in children and adults and giving them aesthetic smiles. It's been quite a rewarding experience. seeing children leaving my clinic with smiles on their faces and seeing their parents beaming, knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards. It's a fun job and I usually am satisfied at the end of the day. Also one of the most highly rewarding specialties in dentistry. But in Nigeria and other parts of the world, I would like you to come visit us one of these days. Thank you for listening.",hello my name is tolu im an orthodontist in a tertiary hospital in nigeria i deal in correcting malocclusions in children and adults and giving them aesthetic smiles its been quite a rewarding experience seeing children leaving my clinic with smiles on their faces and seeing their parents beaming knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards its a fun job and i usually am satisfied at the end of the day also one of the most highly rewarding specialties in dentistry but in nigeria and other parts of the world i would like you to come visit us one of these days thank you for listening,tolu_nigeria_nigeria
2,2024-02-28 00:12:58.756045,32896923-f849-4aa3-9eda-dccc7284a309,/data4/data/prod2/blob_312.wav,1.0,05f2bfd7-822d-4658-ae31-eaa078438fb5,Production-Test-Set,1105,1105,"Hi, what are you doing today?",NaN,...,NaN,NaN,NaN,['today'],"[{""entity"": ""B-DATE"", ""score"": 0.9999139308929443, ""index"": 7, ""word"": ""\u2581today"", ""start"": 23, ""end"": 28}]","[{""entity_group"": ""DATE"", ""score"": 0.9999139308929443, ""word"": ""today"", ""start"": 23, ""end"": 28}]",NaN,"Hi, what are you doing today?",hi what are you doing today,today
8,2024-02-28 00:12:58.721021,31465167-a293-4e13-ab6f-d566ab427c1c,/data4/data/prod2/b

In [117]:
# All the pretrained model results are stored here

def load_models(pred_csv, df_test):
    pred_df = pd.read_csv(pred_csv)
    print(pred_df.columns)
    #pred_df = pred.df.rename("")
    columns=["audio_id", "transcript", "cat_entities"]
    df_merge = pd.merge(df_test[columns], pred_df[["audio_id", "pred_clean" ]], on="audio_id")
    return df_merge

In [150]:
df_test

,date,audio_id,audio_path,audio_duration,sentence_id,source,doc_id,seq_id,text,prediction,...,PER,LOC,ORG,DATE,entities,entities_group,entities_others,transcript_original,transcript,cat_entities
0,2024-02-28 00:12:58.764668,513ef5d4-a823-4113-b757-4ed91d5d5e17,/data4/data/prod2/blob_52537.wav,1.0,7e3ff9d7-79fe-4eba-9c0b-d5c10074a2f5,Production-Test-Set,1107,1107,"Hello, my name is Tolu. I'm an orthodontist in a tertiary hospital in Nigeria. I deal in correcting malocclusions in children and adults and giving them aesthetic smiles. It's been quite a rewarding experience. seeing children leaving my clinic with smiles on their faces and seeing their parents beaming, knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards. It's a fun job and I usually am satisfied at the end of the day. Also one of the most highly rewarding specialties in dentistry. But in Nigeria and other parts of the world, I would like you to come visit us one of these days. Thank you for listening.",NaN,...,['Tolu'],"['Nigeria', 'Nigeria']",NaN,NaN,"[{""entity"": ""B-PER"", ""score"": 0.9999862909317017, ""index"": 6, ""word"": ""\u2581To"", ""start"": 18, ""end"": 20}, {""entity"": ""I-PER"", ""score"": 0.9974403381347656, ""index"": 7, ""word"": ""lu"", ""start"": 20, ""end"": 22}, {""entity"": ""B-LOC"", ""score"": 0.9999977350234985, ""index"": 23, ""word"": ""\u2581Nigeria"", ""start"": 70, ""end"": 77}, {""entity"": ""B-LOC"", ""score"": 0.9999985694885254, ""index"": 141, ""word"": ""\u2581Nigeria"", ""start"": 589, ""end"": 596}]","[{""entity_group"": ""PER"", ""score"": 0.9987133145332336, ""word"": ""Tolu"", ""start"": 18, ""end"": 22}, {""entity_group"": ""LOC"", ""score"": 0.9999977350234985, ""word"": ""Nigeria"", ""start"": 70, ""end"": 77}, {""entity_group"": ""LOC"", ""score"": 0.9999985694885254, ""word"": ""Nigeria"", ""start"": 589, ""end"": 596}]",NaN,"Hello, my name is Tolu. I'm an orthodontist in a tertiary hospital in Nigeria. I deal in correcting malocclusions in children and adults and giving them aesthetic smiles. It's been quite a rewarding experience. seeing children leaving my clinic with smiles on their faces and seeing their parents beaming, knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards. It's a fun job and I usually am satisfied at the end of the day. Also one of the most highly rewarding specialties in dentistry. But in Nigeria and other parts of the world, I would like you to come visit us one of these days. Thank you for listening.",hello my name is tolu im an orthodontist in a tertiary hospital in nigeria i deal in correcting malocclusions in children and adults and giving them aesthetic smiles its been quite a rewarding experience seeing children leaving my clinic with smiles on their faces and seeing their parents beaming knowing that options that were previously unavailable to them are now available to them to help improve the physical appearances of their wards its a fun job and i usually am satisfied at the end of the day also one of the most highly rewarding specialties in dentistry but in nigeria and other parts of the world i would like you to come visit us one of these days thank you for listening,tolu_nigeria_nigeria
2,2024-02-28 00:12:58.756045,32896923-f849-4aa3-9eda-dccc7284a309,/data4/data/prod2/blob_312.wav,1.0,05f2bfd7-822d-4658-ae31-eaa078438fb5,Production-Test-Set,1105,1105,"Hi, what are you doing today?",NaN,...,NaN,NaN,NaN,['today'],"[{""entity"": ""B-DATE"", ""score"": 0.9999139308929443, ""index"": 7, ""word"": ""\u2581today"", ""start"": 23, ""end"": 28}]","[{""entity_group"": ""DATE"", ""score"": 0.9999139308929443, ""word"": ""today"", ""start"": 23, ""end"": 28}]",NaN,"Hi, what are you doing today?",hi what are you doing today,today
8,2024-02-28 00:12:58.721021,31465167-a293-4e13-ab6f-d566ab427c1c,/data4/data/prod2/b

In [151]:
#df_test.to_csv("/data3/abraham/asr_benchmarking/data/intron_fresh_audio_Production-Test-Set-Quality_2024_03_05_21_16_28_africa_ner.csv", index=False)

In [137]:
pred_csv = "/data3/abraham/asr_benchmarking/results/intron-open-test--wav2vec2_large_robust_6m_may24_normal_lr_ep1_3e4_17500-45000-2500_no_inf-checkpoints-checkpoint-45000-prod2-wer-0.4629-962_full.csv"
#pred_csv = "/data3/abraham/asr_benchmarking/results/intron-open-test-parakeet_6m_e2_vocab_replacement_restart_at_5epochs_2epochs-2024-06-08_09-17-18-checkpoints-ASR-Model-Language-en.nemo-prod2-wer-0.3461-962_full.csv"

In [138]:
df_merge = load_models(pred_csv, df_test)


Index(['date', 'audio_id', 'audio_path', 'audio_duration', 'sentence_id',
       'source', 'doc_id', 'seq_id', 'text', 'prediction', 'char_len',
       'quality', 'project_id', 'project_name', 'user_id', 'level', 'status',
       'age_group', 'gender', 'accent', 'discipline', 'education',
       'institution', 'clinical_experience', 'city', 'country',
       'num_pos_feedback_received', 'num_neg_feedback_received',
       'num_feedback_received', 'neg_percent', 'Background Noise',
       'Multiple Speakers', 'Background Speakers',
       'Bad Network or Corrupt Audio', 'Numbers',
       'Leading or Trailing noise or silence', 'Good Quality Audio',
       'African Named Entities', 'Clear Speech', 'Incorrect Transcript',
       'Music', 'Good Quality', 'wer', 'ref_clean', 'pred_clean'],
      dtype='object')


In [139]:
def exact_pred_entities(row, count=False):
    predicted_text = row["pred_clean"]
    predicted_entities = []
    
    if not type(predicted_text) != str:
        entities = row[cat_entities].split(JOIN_CHAR)
        for entity in entities:
            ner_present = len(re.findall(f"\\b{entity}\\b", predicted_text, re.IGNORECASE)) >= 1
            if ner_present:
                predicted_entities.append(entity)
    if count:
        return len(predicted_entities)
    else:
        return f"{JOIN_CHAR}".join(predicted_entities)

In [140]:
def get_most_similar_word(entity, prediction):
    similarity_scores = []
    
    predicted_words = prediction.split(" ")
    for word in predicted_words:
        
        find_word = re.findall(r"\w+", word)
        if len(find_word) == 0:
            continue
        elif len(find_word[0])  <= 1:
            continue
        similarity_score = (word, similar(entity, word))
        similarity_scores.append(similarity_score)
    
    
    if len(similarity_scores) != 0:
        most_similar_word = max(similarity_scores, key=lambda x: x[1])
    else:
        most_similar_word = ("", 0)
        
    return most_similar_word

In [141]:
from nltk import everygrams
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


def medtextalign(entity, prediction, ngram_per_entity=3):
    prediction_n_gram = list(everygrams(prediction.split(), 1, 
                                        ngram_per_entity*len(entity.split())))
    
    result = []
    for ngram in prediction_n_gram:
        ngram_join = " ".join(ngram)
        score = similar(entity, ngram_join)
        score = round(score, 2)

        if score >= 0.5:
            result.append((ngram_join , score))

        if score == 1.0:
            break
    
    all_candidate = pd.DataFrame(result, columns=["pred_entity", "score"])
    best_candidate = all_candidate[all_candidate["score"] == all_candidate["score"].max()]
    best_candidate["ngram"] = best_candidate["pred_entity"].apply(lambda x: len(x.split()))
    best_candidate = best_candidate.sort_values(["score", "ngram"], ascending=True).head(1).reset_index(drop=True)

    return best_candidate, all_candidate

def f(row):
    cat_entities = row["cat_entities"].split(JOIN_CHAR)
    prediction = clean_text(row["pred_clean"])
    
    out = []
    for entity in cat_entities:
        best_, all_ = medtextalign(entity, prediction, ngram_per_entity=3)
        if len(best_)!=0:
            out.append(best_["pred_entity"][0])
    return f"{JOIN_CHAR}".join(out)

In [142]:
df_merge["pred_cat_entities_medtextalign"] = df_merge.apply(lambda x: f(x), axis=1)

In [143]:
df_merge["pred_cat_entities_medtextalign"]

0      tolu_nigeria_nigeria     
1      today                    
2      charles_charles_two weeks
3      meetingday               
4      intron_morning           
            ...                 
528    8yearold                 
529    year old                 
530    80yearold                
531                             
532    8 yearold                
Name: pred_cat_entities_medtextalign, Length: 533, dtype: object

In [144]:
df_merge.columns

Index(['audio_id', 'transcript', 'cat_entities', 'pred_clean',
       'pred_cat_entities_medtextalign'],
      dtype='object')

In [145]:
import jiwer
df_merge["medical_wer"] = df_merge.apply(
            lambda row: jiwer.wer(row["cat_entities"].replace("_", " "), row['pred_cat_entities_medtextalign'].replace("_", " ")), axis=1
        )
medical_wer = jiwer.wer(list(df_merge["cat_entities"].str.replace("_", " ")), list(df_merge['pred_cat_entities_medtextalign'].str.replace("_", " ")))
medical_wer

0.6111111111111112

In [146]:
#0.5240926799261841

In [147]:
df_merge["cat_entities"].str.replace("_", " ")

0      tolu nigeria nigeria     
1      today                    
2      charles charles two weeks
3      today                    
4      intron morning           
            ...                 
528    80 year old              
529    8-year-old               
530    80 year old              
531    80 year old              
532    80 year old              
Name: cat_entities, Length: 533, dtype: object

In [148]:
df_merge["pred_cat_entities_medtextalign"].str.replace("_", " ")

0      tolu nigeria nigeria     
1      today                    
2      charles charles two weeks
3      meetingday               
4      intron morning           
            ...                 
528    8yearold                 
529    year old                 
530    80yearold                
531                             
532    8 yearold                
Name: pred_cat_entities_medtextalign, Length: 533, dtype: object

In [149]:
import jiwer
df_merge["wer"] = df_merge.apply(
            lambda row: jiwer.wer(row["pred_clean"], row[r'transcript']), axis=1
        )
normal_wer = jiwer.wer(list(df_merge["pred_clean"]), list(df_merge[r'transcript']))
normal_wer

0.42566067572910016